In [ ]:
# If run this file on google colab:
# Please use GPU for colab as CPU runtime would make the trainig process extremely slow
# It can be found in Notebook settings or Runtime > Change runtime type
# select GPU as Hardware accelerator.
# There are a limit for using GPU on google colab for free in a period of time
# but the quota is enough just to tryout this notebook

import h5py
import random
import numpy as np
import tensorflow as tf
import pandas as pd
import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG19
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, Input, concatenate
from sklearn.model_selection import train_test_split
import cv2

# Check the tensorflow version. We use tensorflow 2.6.0 in this example,
# but the script should work for all tensorflow 2.x versions.
print(tf.__version__)

In [ ]:
# if using google colab, run the following lines
from google.colab import drive
import os

# give colab the right to access your file
drive.mount('/content/drive')
os.chdir('/content/drive/')

# Normalization

In [ ]:
def normalize(specs):
  '''
  Linear normalization of the data
  @param: specs is the list of spcetrograms to normalize
  @return: the normalized spectrograms
  '''

  return_specs = []
  for i in range(len(specs)):
    # make a copy to ensure not changing the original spectrogram
    cur_spec = np.copy(specs[i])
    s_min = np.amin(cur_spec)
    s_max = np.amax(cur_spec)
    return_specs.append((cur_spec - s_min)/(s_max - s_min) * 255)

  return np.array(return_specs)

# Augmentation Methods

For each function, make a copy of the original spectrogram
to ensure that we do not change the original one

Return all the augmented spectrograms in lists for consistency

In [ ]:
def time_chop(spec, rand_start):
  '''
  chop the spectrogram on x axis (time) from the right
  @param: spec, the spectrogram to chop
  @param: rand_start: the randomed index to start chopping
  @return: the list of augmented spectrograms
  '''

  time_chopped_spec = np.copy(spec)
  time_chopped_spec[:,224 - rand_start:,:] = 0

  return [time_chopped_spec]

def freq_chop(spec, rand_start):
  '''
  chop the spectrogram on y axis (frequency) from the top
  @param: spec, the spectrogram to chop
  @param: rand_start: the randomed index to start chopping
  @return: the list of augmented spectrograms
  '''

  freq_chopped_spec = np.copy(spec)
  freq_chopped_spec[0:rand_start,:,:] = 0

  return [freq_chopped_spec]

def four_chop(spec, rand_start):
  '''
  chop the spectrogram on four sides
  @param: spec, the spectrogram to chop
  @param: rand_start: the randomed index to start chopping
  @return: the list of augmented spectrograms
  '''

  four_chopped_spec = np.copy(spec)
  four_chopped_spec[0 : rand_start,:,:] = 0  # top
  four_chopped_spec[:,224 - rand_start:,:] = 0  # right
  four_chopped_spec[224 - rand_start:,:,:] = 0  # bottom
  four_chopped_spec[:, 0 : rand_start ,:] = 0  # left

  return [four_chopped_spec]

In [ ]:
def add_noises(spec):
  '''
  add noise to the spectrogram with 1/3 ratio
  @param: spec, the spectrogram to chop
  @return: the list of augmented spectrograms
  '''
  # add noise from light rian -2, rain -3, heavy rain -4, thunder -5, aircraft -6, chainsaw -7, and car/truck -8
  return_specs = []
  noise_sonos = [-2, -3, -4, -5,-6,-7,-8]

  for i in range(len(noise_sonos)):
    noises_index = np.argwhere(sonotypes_h5 == noise_sonos[i]).flatten()
    noises = specs_h5[noises_index]
    # randomly pick a noise sample
    index = random.randint(0, len(noises) - 1)
    # normalize sound and noise, add them together with 1/3 ratio
    noise = normalize(np.array(noises[index]) / 3)
    return_specs.append(np.add(normalize([np.copy(spec)])[0], noise))

  return return_specs

In [ ]:
def translate(spec, roll_start):
  '''
  roll the spectrogram up and down
  @param: spec, the spectrogram to chop
  @param: roll_start, the index to start rolling
  @return: the list of augmented spectrograms
  '''

  return_specs = []
  return_specs.append(np.roll(spec, -roll_start, axis = 0))
  return_specs.append(np.roll(spec, roll_start, axis = 0))

  return return_specs

In [ ]:
def widen(spec, widen_index):
  '''
  widen the spectrogram
  @param: spec, the spectrogram to chop
  @param: widen_index, the index to decide the start and end of
          the spectrogram to widen
  @return: the list of augmented spectrograms
  '''
  return_specs = []
  widen_time_spec=cv2.resize(spec.astype('float32'),(224 + widen_index,224))
  widen_freq_spec=cv2.resize(spec.astype('float32'),(224,224 + widen_index))

  return_specs.append(widen_time_spec[:,widen_index // 2: -widen_index // 2,:])
  return_specs.append(widen_freq_spec[widen_index // 2: -widen_index // 2,:,:])

  return return_specs

def squeeze(spec, squeeze_index):
  '''
  squeeze the spectrogram
  @param: spec, the spectrogram to chop
  @param: widen_index, the index to decide the start and end of
          the spectrogram to widen
  @return: the list of augmented spectrograms
  '''

  squeezed=cv2.resize(spec.astype('float32'),(224 - squeeze_index,224 - squeeze_index))
  squeeze_spec = np.zeros([224, 224, 3])
  squeeze_spec[squeeze_index//2 : - squeeze_index //2,squeeze_index//2 : - squeeze_index //2, :] = squeezed

  return [squeeze_spec]

In [ ]:
def augment(specs, aux_input, sonotypes, aug_num, augment_range = 0.1):
  '''
  call all the augment methods on the spectrograms

  @param: specs is the list of spectrograms to augment from
  @param: aux_input is the list of auxiliary input corresponds to the spectrograms
  @param: sonotypes is the list of sonnotypes corresponds to the spectrograms
  @param: aug_num is the number of sets of augmented spectrograms
          (returned number of samples will be 1 + 15*aug_num)
  @param: augment_range is the threshold used for augmentations, default to 0.1
  @return: augment_specs_func is the list of augmented spectrograms
  @return: augment_aux_func is the list of  auxiliary input corresponds to the spectrograms
  @return: augment_sono_func is the list of sonotypes input corresponds to the spectrograms
  '''

  augment_specs_func = []
  augment_aux_func = []
  augment_sono_func = []

  # print(len(aux_input))
  for i in range(len(specs)):
    # generate random index array for augmentation
    # in 5% to 10% of the size of the original spectrogram
    # 224 * 224 is the image size
    indices = np.arange(int(224 * augment_range / 3 * 2) , int(224 * augment_range))
    np.random.shuffle(indices)
    indices = indices[:aug_num]

    # augment each spec and add to list
    cur_spec = np.copy(specs[i])
    # add itself to the list
    if (len(augment_specs_func)):
      augment_specs_func = np.append(augment_specs_func, [cur_spec], axis = 0)
    else:
      augment_specs_func.append(cur_spec)
    # augment_specs_func.append(cur_spec)

    for index in indices:
      # print(index)
      # chop
      augment_specs_func = np.append(augment_specs_func, time_chop( np.copy(cur_spec), index), axis = 0)
      augment_specs_func = np.append(augment_specs_func, freq_chop( np.copy(cur_spec), index), axis = 0)
      augment_specs_func = np.append(augment_specs_func, four_chop( np.copy(cur_spec), index), axis = 0)

      # widen + squeeze
      augment_specs_func = np.append(augment_specs_func, squeeze( np.copy(cur_spec), index), axis = 0)
      augment_specs_func = np.append(augment_specs_func, widen( np.copy(cur_spec), index), axis = 0)

      # noise
      augment_specs_func = np.append(augment_specs_func, add_noises(np.copy(cur_spec)), axis = 0)

      # translate
      augment_specs_func = np.append(augment_specs_func, translate(np.copy(cur_spec), index), axis = 0)

    # total 1 + 15 * aug_num augmented, repeat the sono and aux
    if (len(augment_aux_func)):
      augment_aux_func = np.append(augment_aux_func, np.repeat([aux_input[i]], 1 + 15 * aug_num, axis = 0), axis= 0)
    else:
      augment_aux_func = np.repeat([aux_input[i]], 1 + 15 * aug_num, axis = 0)

    augment_sono_func = np.append(augment_sono_func, np.repeat(sonotypes[i], 1 + 15 * aug_num), axis= 0)

  return augment_specs_func, augment_aux_func, augment_sono_func

In [ ]:
def augment_random_one(specs, aux_input, sonotypes, augment_range = 0.1, repeat_num = 1):
  '''
  Randomly call one of the augment methods on the spectrograms and generate
  repeat_num extra sample for each sample

  @param: specs is the list of spectrograms to augment from
  @param: aux_input is the list of auxiliary input corresponds to the spectrograms
  @param: sonotypes is the list of sonnotypes corresponds to the spectrograms
  @param: augment_range is the threshold used for augmentations, default to 0.1
  @param: repeat_num times to repeat random augmentation process
  @return: augment_specs_func is the list of augmented spectrograms
  @return: augment_aux_func is the list of  auxiliary input corresponds to the spectrograms
  @return: augment_sono_func is the list of sonotypes input corresponds to the spectrograms
  '''
  augment_specs_func = []
  augment_aux_func = []
  augment_sono_func = []

  augment_functions = [time_chop, freq_chop, four_chop, squeeze, widen, add_noises, translate]
  indices = np.arange(int(224 * augment_range / 3 * 2) , int(224 * augment_range))

  for i in range(len(specs)):
    for _ in range(repeat_num):
      cur_spec = np.copy(specs[i])
      selected_function = random.choice(augment_functions)
      if selected_function.__name__ == "add_noises":
        cur_augment_specs = selected_function(cur_spec)
      else:
        # Generate index for augmentation
        index = random.choice(indices)
        cur_augment_specs = selected_function(cur_spec, index)

      cur_augment_spec = random.choice(cur_augment_specs)
      if (len(augment_specs_func)):
        augment_specs_func = np.append(augment_specs_func, [cur_augment_spec], axis = 0)
      else:
        augment_specs_func.append(cur_augment_spec)

    if (len(augment_aux_func)):
      augment_aux_func = np.append(augment_aux_func, np.repeat([aux_input[i]], repeat_num, axis = 0), axis= 0)
    else:
      augment_aux_func = np.repeat([aux_input[i]], repeat_num, axis = 0)

    augment_sono_func = np.append(augment_sono_func,  np.repeat(sonotypes[i], repeat_num), axis= 0)

  return augment_specs_func, augment_aux_func, augment_sono_func

# Read dataset and randomly select samples with augmentation

In [ ]:
f =  h5py.File('Shareddrives/personal/samples.hdf5', "r")

specs_h5 = np.array(f["specs"]).astype("float32")
sonotypes_h5 = np.array(f["sonotypes"]).astype("float32")
times_h5 = np.array(f["times"]).astype("float32")
freqs_h5 = np.array(f["freqs"]).astype("float32")
groups_h5 = np.array(f["groups"])

f.close()

In [ ]:
# append x_times an x_freqs to be auxiliary_input
aux_input_h5 = np.append(times_h5, freqs_h5, axis=1)

# create the dictionary for sonotypes and groups
sono2group = dict(zip(sonotypes_h5, groups_h5))

# get the data for top k sonotypes
s_unique, s_freq = np.unique(sonotypes_h5, return_counts=True)
s_freq_order = np.argsort(s_freq)[::-1]
s_freq_desc = s_freq[s_freq_order]

numUsed = 6
num_pick = 49

In [ ]:
def get_samples(numUsed, min_type, aug_goal = 250, fixed_size = False):
    # get groups
    type_index = np.argwhere((s_freq_desc >= min_type) & (
        s_unique[s_freq_order] > 0)).flatten()
    random.shuffle(type_index)
    type_index = np.sort(type_index[:numUsed])
    max_num = s_freq_desc[np.min(type_index)]
    typeUsed = s_unique[s_freq_order][type_index]

    print("type index:", type_index)
    print("type used: ", typeUsed)
    print("max num: ", max_num)

    # aug
    specs = []
    aux_input = []
    sonotypes = []
    spec_test = []
    aux_test = []
    y_test = []
    spec_val = []
    aux_val = []
    y_val = []
    sizes = []

    # no aug
    specs_no = []
    aux_input_no = []
    sonotypes_no = []
    spec_test_no = []
    aux_test_no = []
    y_test_no = []
    spec_val_no = []
    aux_val_no = []
    y_val_no = []

    for i in range(len(typeUsed)):
        # get index of the current type of spec
        cur_index = np.argwhere(sonotypes_h5 == typeUsed[i]).flatten()
        random.shuffle(cur_index)
        # fixed size, comment off next line if use the original size
        if fixed_size:
          cur_index = cur_index[:min_type]
        sizes.append(len(cur_index))  # append the used size

        # decide the text and val size, at least 1 for both test and val
        text_val_size = max(1, int(len(cur_index) * 0.1))
        cur_index_resized = cur_index[:len(cur_index) - 2 * text_val_size]
        test_index = cur_index[len(cur_index) - 2 *
                               text_val_size: len(cur_index) - text_val_size]
        val_index = cur_index[len(cur_index) - text_val_size: len(cur_index)]
        print("\nsonotype, len of cur:", typeUsed[i], len(cur_index))
        print("train, test, val size: ", len(cur_index_resized),
              len(test_index), len(val_index))

        # augment to aug goal
        # train
        if int(aug_goal * 0.8) > len(cur_index_resized):
          augment_repeat_cnt = int(aug_goal * 0.8) // len(cur_index_resized)
          augment_specs, augment_aux, augment_sono = augment_random_one(
                        specs_h5[cur_index_resized],
                        aux_input_h5[cur_index_resized],
                        np.repeat(i, len(cur_index_resized)),
                        repeat_num = augment_repeat_cnt
                        )
        else:
          augment_specs, augment_aux, augment_sono =  specs_h5[cur_index_resized], aux_input_h5[cur_index_resized], np.repeat(i, len(cur_index_resized)),

        aug_index = np.arange(len(augment_specs))
        random.shuffle(aug_index)
        aug_index = aug_index[:int(aug_goal * 0.8) - len(cur_index_resized)]
        print("test aug size", len(aug_index))

        if len(specs):
            specs = np.concatenate(
                (specs, specs_h5[cur_index_resized], augment_specs[aug_index]), axis=0)
            aux_input = np.concatenate(
                (aux_input, aux_input_h5[cur_index_resized], augment_aux[aug_index]), axis=0)
            sonotypes = np.append(sonotypes, np.repeat(
                i, len(cur_index_resized) + len(aug_index)))

            specs_no = np.concatenate(
                (specs_no, specs_h5[cur_index_resized]), axis=0)
            aux_input_no = np.concatenate(
                (aux_input_no, aux_input_h5[cur_index_resized]), axis=0)
            sonotypes_no = np.append(sonotypes_no, np.repeat(
                i, len(cur_index_resized)))
        else:
            specs = np.concatenate(
                (specs_h5[cur_index_resized], augment_specs[aug_index]), axis=0)
            aux_input = np.concatenate(
                (aux_input_h5[cur_index_resized], augment_aux[aug_index]), axis=0)
            sonotypes = np.repeat(i, len(cur_index_resized) + len(aug_index))

            specs_no = np.copy(specs_h5[cur_index_resized])
            aux_input_no = np.copy(aux_input_h5[cur_index_resized])
            sonotypes_no = np.repeat(i, len(cur_index_resized))

        # test and val
        # augment_num = (int(aug_goal * 0.1) // text_val_size) // 16 + 1
        augment_repeat_cnt = int(aug_goal * 0.1) // len(test_index)
        augment_specs, augment_aux, augment_sono = None, None, None
        # test
        if int(aug_goal * 0.1) > len(test_index):
          augment_repeat_cnt = int(aug_goal * 0.1) // len(test_index)
          augment_specs, augment_aux, augment_sono = augment_random_one(
                        specs_h5[test_index],
                        aux_input_h5[test_index],
                        np.repeat(i, len(test_index)),
                        repeat_num = augment_repeat_cnt
                        )
        else:
          augment_specs, augment_aux, augment_sono =  specs_h5[test_index], aux_input_h5[test_index], np.repeat(i, len(test_index))

        aug_index = np.arange(len(augment_specs))
        random.shuffle(aug_index)
        aug_index = aug_index[:int(aug_goal * 0.1) - text_val_size]
        print("test aug size", len(aug_index))

        if len(spec_test):
            spec_test = np.concatenate(
                (spec_test, specs_h5[test_index], augment_specs[aug_index]), axis=0)
            aux_test = np.concatenate(
                (aux_test, aux_input_h5[test_index], augment_aux[aug_index]), axis=0)
            y_test = np.append(y_test, np.repeat(
                i, len(test_index) + len(aug_index)))

            spec_test_no = np.concatenate(
                (spec_test_no, specs_h5[test_index]), axis=0)
            aux_test_no = np.concatenate(
                (aux_test_no, aux_input_h5[test_index]), axis=0)
            y_test_no = np.append(y_test_no, np.repeat(i, len(test_index)))

        else:
            spec_test = np.concatenate(
                (specs_h5[test_index], augment_specs[aug_index]), axis=0)
            aux_test = np.concatenate(
                (aux_input_h5[test_index], augment_aux[aug_index]), axis=0)
            y_test = np.repeat(i, len(test_index) + len(aug_index))

            spec_test_no = np.copy(specs_h5[test_index])
            aux_test_no = np.copy(aux_input_h5[test_index])
            y_test_no = np.repeat(i, len(test_index))

        # val
        augment_repeat_cnt = int(aug_goal * 0.1) // len(val_index)
        augment_specs, augment_aux, augment_sono = None, None, None
        # test
        if int(aug_goal * 0.1) > len(val_index):
          augment_repeat_cnt = int(aug_goal * 0.1) // len(val_index)
          augment_specs, augment_aux, augment_sono = augment_random_one(
                        specs_h5[val_index],
                        aux_input_h5[val_index],
                        np.repeat(i, len(val_index)),
                        repeat_num = augment_repeat_cnt
                        )
        else:
          augment_specs, augment_aux, augment_sono =  specs_h5[val_index], aux_input_h5[val_index], np.repeat(i, len(val_index))

        aug_index = np.arange(len(augment_specs))
        random.shuffle(aug_index)
        aug_index = aug_index[:int(aug_goal * 0.1) - text_val_size]

        if len(spec_val):
            spec_val = np.concatenate(
                (spec_val, specs_h5[val_index], augment_specs[aug_index]), axis=0)
            aux_val = np.concatenate(
                (aux_val, aux_input_h5[val_index], augment_aux[aug_index]), axis=0)
            y_val = np.append(y_val, np.repeat(
                i, len(val_index) + len(aug_index)))

            spec_val_no = np.concatenate(
                (spec_val_no, specs_h5[val_index]), axis=0)
            aux_val_no = np.concatenate(
                (aux_val_no, aux_input_h5[val_index]), axis=0)
            y_val_no = np.append(y_val_no, np.repeat(i, len(val_index)))
        else:
            spec_val = np.concatenate(
                (specs_h5[val_index], augment_specs[aug_index]), axis=0)
            aux_val = np.concatenate(
                (aux_input_h5[val_index], augment_aux[aug_index]), axis=0)
            y_val = np.repeat(i, len(val_index) + len(aug_index))

            spec_val_no = np.copy(specs_h5[val_index])
            aux_val_no = np.copy(aux_input_h5[val_index])
            y_val_no = np.repeat(i, len(val_index))


        print("Cur sizes for validation", end=":")
        print(specs.shape, len(aux_input), len(sonotypes), len(spec_test), len(
            aux_test), len(y_test), len(spec_val), len(aux_val), len(y_val))

    # aug
    specs = normalize(specs)
    x_test = [normalize(spec_test), aux_test]
    x_val = [normalize(spec_val), aux_val]
    cat_y_test = to_categorical(pd.factorize(
        y_test)[0], num_classes=len(typeUsed))
    cat_y_val = to_categorical(pd.factorize(
        y_val)[0], num_classes=len(typeUsed))

    # no aug
    specs_no = normalize(specs_no)
    x_test_no = [normalize(spec_test_no), aux_test_no]
    x_val_no = [normalize(spec_val_no), aux_val_no]
    cat_y_test_no = to_categorical(pd.factorize(
        y_test_no)[0], num_classes=len(typeUsed))
    cat_y_val_no = to_categorical(pd.factorize(
        y_val_no)[0], num_classes=len(typeUsed))

    print("train, test, val size:", specs.shape[0], len(
        cat_y_test), len(cat_y_val))

    return typeUsed, sizes, specs, aux_input, sonotypes, x_test, x_val, cat_y_test, cat_y_val,specs_no,aux_input_no, sonotypes_no, x_test_no, x_val_no, cat_y_test_no, cat_y_val_no, y_test_no, y_test

In [ ]:
typeUsed, sizes, specs, aux_input, sonotypes, x_test, x_val, cat_y_test, cat_y_val,specs_no,aux_input_no, sonotypes_no, x_test_no, x_val_no, cat_y_test_no, cat_y_val_no, y_test_no, y_test = get_samples(
    numUsed, num_pick)

# Model and Training

In [ ]:
# Feel free to play with different configs.
config = dict(
    dropout = 0.5,
    hidden = 1024,
    learn_rate = 0.00001,
    epochs = 300,
    )

In [ ]:
def build_finetune_model(base_model, dropouts, fc_layers, num_classes):
    '''
    finetune the model, freeze teh top layers,
    add dropouts, dense layers,
    another input layer for auxiliary input
    and concatenate it with the flatten layer
    '''

    # freeze the base layers
    for layer in base_model.layers:
       layer.trainable = False

    # add flatten layer
    x = base_model.output
    x = Flatten()(x)

    # add input layer for auxiliary input (time and frequency)
    auxiliary_input = Input(shape=(4,), name='aux_input')
    x = concatenate([x, auxiliary_input])

    #  dense and dropout layer
    for fc, drop in zip(fc_layers, dropouts):
        x = Dense(fc, activation='relu')(x)
        x = Dropout(drop)(x)

    # final dense layer for output
    predictions = Dense(num_classes, activation='softmax')(x)

    finetune_model = Model(inputs=[base_model.input,auxiliary_input], outputs=predictions)

    return finetune_model

In [ ]:
class TestCallback(keras.callbacks.Callback):
    '''
    The class used to see the test result during training
    '''

    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

In [ ]:
model = None
keras.backend.clear_session()
# get the pretrained model
model = VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3))
# finetune to our case
model = build_finetune_model(model,
                             [config["dropout"], config["dropout"]],
                             [config["hidden"], config["hidden"]],
                             len(typeUsed))

In [ ]:
# # model stats, remove "#" in lines below to print
# model.summary()
# plot_model(model,to_file = 'file-to-store-png.png')

In [ ]:
# earlystopping, checkpoiint, before
filepath_loss = 'Shareddrives/personal/model_loss.hdf5'

earlystop = EarlyStopping(
    monitor='val_loss', mode='min', verbose=1, patience=15)

checkpoint = ModelCheckpoint(filepath_loss, monitor='val_loss', verbose=1,
                              save_best_only=True, save_weights_only=False, mode='auto', save_freq="epoch")
opt = Adam(learning_rate=config["learn_rate"])
model.compile(optimizer=opt, loss='categorical_crossentropy',
              metrics=['accuracy'])
cat_y_train = to_categorical(sonotypes, num_classes=len(typeUsed))

# training
history = model.fit(x=[specs, aux_input], y=cat_y_train, validation_data=(
    x_val, cat_y_val), epochs=config["epochs"], verbose=2, callbacks=[checkpoint, earlystop, TestCallback((x_test, cat_y_test))])

# If we do not want to earlystop, use the next line for training. The model might
# be overfitting but the saved model (to filepath_loss) was controlled by using
# the model with the lowest validation loss, which does not overfit to the
# training samples.
history = model.fit(x=[specs, aux_input], y=cat_y_train, validation_data=(
    x_val, cat_y_val), epochs=config["epochs"], verbose=2, callbacks=[checkpoint, TestCallback((x_test, cat_y_test))])

# Evaluation

In [ ]:
# # Evaluate the model
# # current model
# results = model.evaluate( x= x_test, y=cat_y_test)
# print("cur test loss, test acc:", results)

# The model with best loss
# load the model
model = None
keras.backend.clear_session()
model = load_model (filepath_loss)

# evaluate
results = model.evaluate( x= x_test, y=cat_y_test)

print("best loss test loss, test acc:", results)

In [ ]:
# ROC/AUC
from sklearn.metrics import roc_auc_score

y_score = model.predict(x_test)
y_class = [typeUsed[i]
            for i in np.argmax(y_score, axis=1)]  # predicted class
y_true = [typeUsed[i] for i in y_test]  # true class
roc_score = roc_auc_score(
    y_test, y_score, sample_weight=None, multi_class="ovo")

print(roc_score)

In [ ]:
# precision, recall, f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

y_predict_index = np.argmax(y_score, axis=1)

precision = precision_score(y_test, y_predict_index, average="macro")
recall = recall_score(y_test, y_predict_index, average="macro")
f1 = f1_score(y_test, y_predict_index, average="macro")

print("precision:", precision)
print("recall:", recall)
print("f1 score:", f1)